In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time


In [108]:
import re
import numpy as np

In [5]:
URL = "https://fortune.com/fortune500/list/"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title data-react-helmet="true">
   Fortune 500 Companies 2017: Who Made the List
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="Fortune.com"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Fortune" data-react-helmet="true" property="og:site_name">
   <meta content="en_US" data-react-helmet="true" property="og:locale">
    <meta content="http://fortune.com/fortune500/list/" data-react-helmet="true" property="og:url">
     <meta content="Fortune 500 Companies 2017: Who Made the List" data-react-helmet="true" property="og:title">
      <meta content="The 2017 ranking of the best in U.S. business has been revealed. See who made the list." data-react-helmet="true" property="og:description">
       <meta content="https://fortunedotcom.files.wordpress.com/2017/05/2520_500.png" data-rea

In [17]:
co_names = soup.find_all("span", attrs= {'class':'column small-5 company-title'})

In [19]:
co_names_str = [tag.get_text() for tag in co_names]
co_names_str

['Walmart',
 'Berkshire Hathaway',
 'Apple',
 'Exxon Mobil',
 'McKesson',
 'UnitedHealth Group',
 'CVS Health',
 'General Motors',
 'AT&T;',
 'Ford Motor',
 'AmerisourceBergen',
 'Amazon.com',
 'General Electric',
 'Verizon',
 'Cardinal Health',
 'Costco',
 'Walgreens Boots Alliance',
 'Kroger',
 'Chevron',
 'Fannie Mae']

Need to scroll down to find companies, so giving up and using a csv from an outside source.

In [22]:
fortune_500_list = pd.read_csv('../Data/fortune500_cos_20170331.csv')

In [23]:
fortune_500_list.head()

,Rank,Company Name,Number of Employees,Previous Rank,Revenues ($millions),Revenue Change,Profits ($millions),Profit Change,Assets ($millions),Market Value As of 3/31/17 ($m)
0,1,Walmart,"2,300,000",1.0,"$485,873",0.8%,"$13,643.0",-7.2%,"$198,825","$218,619"
1,2,Berkshire Hathaway,"367,700",4.0,"$223,604",6.1%,"$24,074.0",0.0%,"$620,854","$411,035"
2,3,Apple,"116,000",3.0,"$215,639",-7.7%,"$45,687.0",-14.4%,"$321,686","$753,718"
3,4,Exxon Mobil,"72,700",2.0,"$205,004",-16.7%,"$7,840.0",-51.5%,"$330,314","$340,056"
4,5,McKesson,"68,000",5.0,"$192,487",6.2%,"$2,258.0",53.0%,"$56,563","$31,439"


In [174]:
company_list = fortune_500_list[['Company Name']]

company_list = company_list.assign(indeed_url= pd.Series(np.nan))
company_list.head()

,Company Name,indeed_url
0,Walmart,NaN
1,Berkshire Hathaway,NaN
2,Apple,NaN
3,Exxon Mobil,NaN
4,McKesson,NaN


In [54]:
#url to scrape with company names: https://www.indeed.com/jobs?q=caterpillar&l=
#replace "caterpillar" with whatever company I want to search, use "+" for spaces, "%27" for an apostrophe, "%2C" for comma

def search_text_parse (strng):
    return strng.replace(' ', '+').replace(",","%2C").replace("'", "%27").replace("&","%26")

search_text_parse("testing the function of this thing's fun")

'testing+the+function+of+this+thing%27s+fun'

In [262]:
def cmp_name(strng):
    return strng.replace(' ', '-').replace(",","").replace(".", "")

cmp_name('testing this thing, with. a text')

'testing-this-thing-with-a-text'

In [249]:
import time
import random

i = 0
#for loop to come
for i in range(len(company_list)):
    
    #if np.isnan(company_list['indeed_url'][i]):
    if not isinstance(company_list.loc[i,'indeed_url'], str):
        
        co_name = company_list['Company Name'][i]
        co_name_parsed = search_text_parse(co_name)

        text = requests.get('https://www.indeed.com/jobs?q=' + co_name_parsed + '&l=').text
        text = BeautifulSoup(text, 'html.parser')

        #<a data-tn-element="companyName">
        company_names = text.find_all('span', {'class': 'company'})
        print(co_name)

        for spans in company_names:
            tag = spans.find('a')
            if tag == None:
                next
            else:
                name = tag.get_text()
                if None != re.match('^'+co_name[0:3], name.strip(), flags = re.I):
                    if tag['href'][1:4] == 'cmp':
                        company_list.loc[i,'indeed_url'] = tag['href']
                        print('did it: ', company_list['indeed_url'][i] )
                        break
                    else: pass

        time.sleep(random.random())
    else: 
        pass
    
company_list




AT&T
AmerisourceBergen
did it:  /cmp/Amerisourcebergen
Amazon.com
did it:  /cmp/Amazon.com
General Electric
did it:  /cmp/GE-Corporate
Verizon
did it:  /cmp/Verizon
Cardinal Health
did it:  /cmp/Cardinal-Health
Costco
did it:  /cmp/Costco-Wholesale
Walgreens Boots Alliance
did it:  /cmp/Walgreens
Kroger
did it:  /cmp/Kroger-Stores
Chevron
did it:  /cmp/Chevron
Fannie Mae
did it:  /cmp/Fannie-Mae
J.P. Morgan Chase
Express Scripts Holding
did it:  /cmp/Express-Scripts
Home Depot
did it:  /cmp/The-Home-Depot
Boeing
did it:  /cmp/Boeing
Wells Fargo
did it:  /cmp/Wells-Fargo
Bank of America Corp.
did it:  /cmp/Bank-of-America
Alphabet
Microsoft
did it:  /cmp/Microsoft
Anthem
did it:  /cmp/Anthem,-Inc.
Citigroup
did it:  /cmp/Citi
Comcast
did it:  /cmp/Comcast
IBM
did it:  /cmp/IBM
State Farm Insurance Cos.
Phillips 66
did it:  /cmp/Phillips-66
Johnson & Johnson
did it:  /cmp/Johnson-Controls
Procter & Gamble
did it:  /cmp/Procter-&-Gamble
Valero Energy
did it:  /cmp/Valero
Target
did it:  /

NRG Energy
did it:  /cmp/Nrg-Energy
VF
did it:  /cmp/Vf-Corporation
Devon Energy
did it:  /cmp/Devon-Energy
D.R. Horton
did it:  /cmp/D.r.-Horton
Bed Bath & Beyond
did it:  /cmp/Bed-Bath-&-Beyond
Consolidated Edison
did it:  /cmp/Consolidated-Chassis-Management
Edison International
did it:  /cmp/Edison-International
Sherwin-Williams
did it:  /cmp/Sherwin--williams
NGL Energy Partners
did it:  /cmp/Ngl-Energy-Partners
Dominion Resources
did it:  /cmp/Dominion-Diagnostics
Ameriprise Financial
did it:  /cmp/Ameriprise-Financial
ADP
did it:  /cmp/ADP
Hilton Worldwide Holdings
did it:  /cmp/Hilton-SAN-Francisco-Airport-Bayfront
First Data
did it:  /cmp/First-Data
Henry Schein
did it:  /cmp/Henry-Schein
Toys “R” Us
did it:  /cmp/Toys-'r'-US
BB&T Corp.
Reinsurance Group of America
Core-Mark Holding
Biogen
did it:  /cmp/Biogen
Las Vegas Sands
did it:  /cmp/Las-Vegas-Sands-Corp.
Stanley Black & Decker
did it:  /cmp/Stanley-Black-&-Decker
Parker-Hannifin
did it:  /cmp/Parker-Hannifin
Stryker
did

Harley-Davidson
did it:  /cmp/Harley--davidson
Regions Financial
did it:  /cmp/Regions-Bank
Intercontinental Exchange
did it:  /cmp/Intercontinental-Exchange
Alaska Air Group
did it:  /cmp/Alaska-Airlines
Old Republic International
did it:  /cmp/Old-Navy
Lam Research
did it:  /cmp/Lam-Research
AK Steel Holding
Rockwell Automation
did it:  /cmp/Rockwell-Automation
Adobe Systems
Avon Products
did it:  /cmp/Avon
Terex
did it:  /cmp/Terex
NVR
did it:  /cmp/Nvr,-Inc
Dana Holding
did it:  /cmp/Dana-Corporation
Realogy Holdings
did it:  /cmp/Realogy
American Tower
did it:  /cmp/American-Airlines
Packaging Corp. of America
did it:  /cmp/Packaging-Corporation-of-America
Citizens Financial Group
did it:  /cmp/City-of-Fort-Collins
United Rentals
did it:  /cmp/United-Rentals,-Inc.
Clorox
did it:  /cmp/The-Clorox-Company
Genesis Healthcare
did it:  /cmp/Genesis-Healthcare
M&T Bank Corp.
Ingredion
did it:  /cmp/Ingredion
UGI
did it:  /cmp/Ugi
Owens Corning
did it:  /cmp/Owens-Corning
S&P Global
Mark

,Company Name,indeed_url
0,Walmart,/cmp/Walmart
1,Berkshire Hathaway,/cmp/Berkshire-Hathaway-Specialty-Insurance
2,Apple,/cmp/Apple
3,Exxon Mobil,/cmp/Exxonmobil
4,McKesson,/cmp/Mckesson
5,UnitedHealth Group,/cmp/UnitedHealth-Group
6,CVS Health,/cmp/CVS-Health
7,General Motors,/cmp/General-Motors
8,AT&T,NaN
9,Ford Motor,/cmp/Ford-Motor-Company


In [254]:
null_list = company_list.loc[company_list.loc[:,'indeed_url'].isnull()]
len(null_list)

87

Need to try to eliminate null values

In [271]:
null_list.loc[:,'indeed_url'] = null_list.loc[:,'Company Name'].apply(cmp_name)
null_list = null_list.loc[:,['Company Name','indeed_url']]
null_list.loc[:,'indeed_url'] = ['/cmp/' + x for x in null_list.loc[:,'indeed_url']]
null_list

,Company Name,indeed_url
8,AT&T,/cmp/AT&T
20,J.P. Morgan Chase,/cmp/JP-Morgan-Chase
26,Alphabet,/cmp/Alphabet
32,State Farm Insurance Cos.,/cmp/State-Farm-Insurance-Cos
44,Archer Daniels Midland,/cmp/Archer-Daniels-Midland
48,Albertsons Cos.,/cmp/Albertsons-Cos
59,Cisco Systems,/cmp/Cisco-Systems
61,Dow Chemical,/cmp/Dow-Chemical
82,United Continental Holdings,/cmp/United-Continental-Holdings
86,TJX,/cmp/TJX


In [288]:

test = pd.merge(test, null_list, how='left', on= 'Company Name', left_on=None, right_on=None, sort=True,
         suffixes=('', '_y'), copy=True, indicator=False)
test.loc[test.loc[:,'indeed_url'].isnull(),'indeed_url'] = test.loc[test.loc[:,'indeed_url'].isnull(), 'indeed_url_y']
test = test.iloc[:,[0,1]]
company_list = test
company_list.isnull().any()

Company Name    False
indeed_url      False
dtype: bool

In [289]:
company_list.to_csv('../Data/fort500_cmp_url.csv')